In [ ]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab]

In [ ]:
from haystack.telemetry import tutorial_running

tutorial_running(21)
from haystack.nodes import PromptNode

In [ ]:
!pip install farm-haystack[faiss]
!pip install farm-haystack[inference]

In [ ]:
import os
from haystack.pipelines.standard_pipelines import TextIndexingPipeline
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import DensePassageRetriever
from haystack.utils import print_documents
from haystack.pipelines import DocumentSearchPipeline
from haystack.nodes import AnswerParser, PromptModel, PromptNode, PromptTemplate
from haystack import Pipeline

document_store = FAISSDocumentStore(embedding_dim=128, faiss_index_factory_str="Flat")
document_store.delete_documents()


In [ ]:
doc_dir="/content/docs"


In [ ]:
retriever = DensePassageRetriever(
  document_store=document_store,
  query_embedding_model="vblagoje/dpr-question_encoder-single-lfqa-wiki",
  passage_embedding_model="vblagoje/dpr-ctx_encoder-single-lfqa-wiki",
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/495 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/494 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
def file_setup(doc_dir):
  files_to_index = [doc_dir + "/" + f for f in os.listdir(doc_dir)]
  indexing_pipeline = TextIndexingPipeline(document_store)
  indexing_pipeline.run_batch(file_paths=files_to_index)
  document_store.update_embeddings(retriever)

file_setup(doc_dir=doc_dir)

prompt_template = PromptTemplate(prompt = """Given the provided Documents, Summarize the retrieved document. output "None" if input not suffcient.\n
                                                Query: {query}\n
                                                Documents: {join(documents)}
                                                Answer:
                                          """,
                                 output_parser=AnswerParser())

prompt_node = PromptNode(model_name_or_path="google/flan-t5-large",default_prompt_template=prompt_template)

genPipe = Pipeline()
genPipe.add_node(component=retriever, name="Retriever", inputs=["Query"])
genPipe.add_node(component=prompt_node, name="prompt_node", inputs=["Retriever"])

Preprocessing: 100%|██████████| 3/3 [00:00<00:00, 168.66docs/s]
Writing Documents: 10000it [00:00, 176259.40it/s]        
Create embeddings:  50%|█████     | 16/32 [00:00<00:00, 54.68 Docs/s]
Documents Processed: 10000 docs [00:00, 25423.80 docs/s]


In [ ]:
def questioning():
  question=input("enter your question")
  result=genPipe.run(query=question, params={"Retriever": {"top_k": 1}})
  print(result)



In [ ]:
questioning()

enter your questionhow will congress ensure employment?
{'answers': [<Answer {'answer': 'Congress will provide fiscal incentives to businesses that employ a certain percentage of women. Exports create jobs. Congress will reward export-oriented industries through tax rebates and incentives. Tourism creates jobs. Congress promises an adequately capitalised Tourism Development Bank to provide low-cost, long-term funds for investment in tourism-related businesses. Congress will also offer lower rates of corporate and personal income tax on tourism-related business income.', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': ['906e384fe59e27049dad1e2c9f776652'], 'meta': {'prompt': "Given the provided Documents, Summarize the retrieved document. output 'None' if input not suffcient.\n\n                                                Query: how will congress ensure employment?\n\n                                      

In [ ]:
questioning()

enter your questionWhy should we select Congress
{'answers': [<Answer {'answer': 'The Indian National Congress has been at the forefront of many such battles, and India has always triumphed. Congress led and won India’s struggle for freedom. We established and nurtured parliamentary democracy. We led India to adopt a visionary Constitution that ushered in peaceful socio-economic transformation, established institutions that safeguarded our Republic, and provided the nation its moral compass that has stood the test of time. On the other hand, under the BJP', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': ['80e538b7f1cb17a2590b683797839826'], 'meta': {'prompt': "Given the provided Documents, Summarize the retrieved document. output 'None' if input not suffcient.\n\n                                                Query: Why should we select Congress\n\n                                                Documents:

In [ ]:
questioning()

enter your questionwhat were the key pledges of congress?
{'answers': [<Answer {'answer': 'Congress promises to end the sense of impunity, stamp out mob violence and lynching, and prevent atrocities and hate crimes against the SC, ST, women and minorities. Congress will hold accountable the police and district administration for proven negligence in the case of riots, mob violence and hate crimes.', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': ['4940104fecce05f13b4e270f4998d3c4'], 'meta': {'prompt': "Given the provided Documents, Summarize the retrieved document. output 'None' if input not suffcient.\n\n                                                Query: what were the key pledges of congress?\n\n                                                Documents: Congress promises to end the sense of impunity, stamp out mob violence and lynching, and prevent atrocities and hate crimes against the SC, ST, women a

In [ ]:
questioning()

enter your questionHow will Congress ensure effective delivery of  government services to citizen
{'answers': [<Answer {'answer': 'Congress will work with State Governments to create new Seva Mitra positions (estimated at 10 lakh) in every Gram Panchayat and every Urban Local Body to ensure the effective delivery of government services to citizens ensuring employment', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': ['3826a35c639e5db3fd96df05b8333d8b'], 'meta': {'prompt': "Given the provided Documents, Summarize the retrieved document. output 'None' if input not suffcient.\n\n                                                Query: How will Congress ensure effective delivery of  government services to citizen\n\n                                                Documents: As a condition for devolution of funds to the healthcare and education sectors and to Panchayats and Municipalities, Congress will request Sta

In [ ]:
questioning()

enter your questionHow will Congress increase the wages of Para state workers
{'answers': [<Answer {'answer': 'To increse the wages for the Para-state workers like Anganwadi workers, ASHA workers, rozgarsahayaks, preraks, anudeshaks and other such groups Congress will increase funding for the relevant programmes and work with State Governments to ensure that they are paid more than minimum wage and all arrears are paid immediately.', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': ['dee5a00affddb5771bf21e52a24953d6'], 'meta': {'prompt': "Given the provided Documents, Summarize the retrieved document. output 'None' if input not suffcient.\n\n                                                Query: How will Congress increase the wages of Para state workers\n\n                                                Documents: To increse the wages for the Para-state workers like Anganwadi workers, ASHA workers, rozgarsaha

In [ ]:
questioning()

enter your questionWhat will Congress eliminate


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'answers': [<Answer {'answer': 'None', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': ['4940104fecce05f13b4e270f4998d3c4'], 'meta': {'prompt': "Given the provided Documents, Summarize the retrieved document. output 'None' if input not suffcient.\n\n                                                Query: What will Congress eliminate\n\n                                                Documents: Congress promises to end the sense of impunity, stamp out mob violence and lynching, and prevent atrocities and hate crimes against the SC, ST, women and minorities. Congress will hold accountable the police and district administration for proven negligence in the case of riots, mob violence and hate crimes.\nCelebrating Freedom\nCongress promises to uphold the values enshrined in the Constitution of India and protect their freedoms, including the freedom to dissent. Congress will pass a law on privacy; restrict the us